<a href="https://colab.research.google.com/github/BhanuPrakash-16/GAN_for_images_Project-3/blob/main/GAN_for_images_Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# =====================================
# Cell 1: Global Configuration
# =====================================

import os
from pathlib import Path

BASE_DIR = Path.cwd()

CONFIG = {
    "img_size": 64,
    "channels": 3,
    "latent_dim": 100,
    "batch_size": 64,
    "epochs": 50,
    "lr": 0.0002,
    "beta_1": 0.5,
    "beta_2": 0.999,
7
    "data_dir": "/content/drive/MyDrive/DCGANProject/ExampleDataDir/PartialPlantExamples",
    "checkpoint_dir": "/content/drive/MyDrive/DCGANProject/checkpoints",
    "sample_dir": "/content/drive/MyDrive/DCGANProject/samples",
    "log_dir": "/content/drive/MyDrive/DCGANProject/logs",
}

for d in ["checkpoint_dir", "sample_dir", "log_dir"]:
    os.makedirs(CONFIG[d], exist_ok=True)
CONFIG


In [ ]:
# =====================================
# Cell 2: Data Configuration Loader (FIXED)
# =====================================

import os
import yaml

os.makedirs(r"/content/drive/MyDrive/DCGANProject\configs", exist_ok=True)

DATA_CONFIG = {
    "image_size": 64,
    "classes": [
        "healthy",
        "early_blight",
        "late_blight",
        "rust",
        "mildew",
        "mosaic"
    ],
    "splits": ["train", "validation", "test"]
}

with open(r"/content/drive/MyDrive/DCGANProject\configs\data_config.yaml", "w") as f:
    yaml.dump(DATA_CONFIG, f)

print("data_config.yaml created successfully inside configs/")

# A YAML file is a human-readable configuration file used to store settings,
# parameters, and structured data without writing code.
# YAML is used to keep paths, hyperparameters, and experiment settings cleanly
# separated from Python code.


In [ ]:
# =====================================
# Cell 3: Dataset Loader (FINAL FIX)
# =====================================
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

DATASET_ROOT = r"/content/drive/MyDrive/DCGANProject/ExampleDataDir/PartialPlantExamples"

def load_plantvillage_dataset(img_size=64, batch_size=64):
    ds = image_dataset_from_directory(
        DATASET_ROOT,
        label_mode=None,              # GAN → no labels
        image_size=(img_size, img_size),
        batch_size=batch_size,
        shuffle=True
    )

    # Normalize images to [-1, 1] for DCGAN
    ds = ds.map(
        lambda x: (tf.cast(x, tf.float32) / 127.5) - 1.0,
        num_parallel_calls=tf.data.AUTOTUNE
    )

    return ds.prefetch(tf.data.AUTOTUNE)

print ("Data Set Root", DATASET_ROOT)

def count_images_in_directory(DATASET_ROOT):
    valid_ext = (".jpg", ".jpeg", ".png", ".bmp")
    count = 0
    for root, _, files in os.walk(DATASET_ROOT):
        count += sum(f.lower().endswith(valid_ext) for f in files)
    return count

total_images = count_images_in_directory(DATASET_ROOT)
print("Total images:", total_images)



In [ ]:
# =====================================
# Cell 4: Generator Model (FIXED)
# =====================================

from tensorflow.keras import layers, models

def build_generator(latent_dim=100):
    model = models.Sequential(name="Generator")

    model.add(layers.Dense(4 * 4 * 512, input_dim=latent_dim))
    model.add(layers.Reshape((4, 4, 512)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # 4x4 → 8x8
    model.add(layers.Conv2DTranspose(256, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # 8x8 → 16x16
    model.add(layers.Conv2DTranspose(128, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # 16x16 → 32x32
    model.add(layers.Conv2DTranspose(64, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # 32x32 → 64x64 ✅ (MISSING EARLIER)
    model.add(layers.Conv2DTranspose(32, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # Final RGB image
    model.add(layers.Conv2DTranspose(
        3, kernel_size=3, activation="tanh", padding="same"
    ))

    return model


# Sanity check
generator = build_generator()
generator.summary()


In [ ]:
# =====================================
# Cell 5: Discriminator Model (With Summary)
# =====================================

from tensorflow.keras import layers, models

def build_discriminator(img_size=64, channels=3):
    model = models.Sequential(name="Discriminator")

    model.add(layers.Conv2D(
        64,
        kernel_size=4,
        strides=2,
        padding="same",
        input_shape=(img_size, img_size, channels)
    ))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, 4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, 4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))

    return model


# ====== Instantiate & Print Summary ======
discriminator = build_discriminator(img_size=64, channels=3)
discriminator.summary()


In [ ]:
# =====================================
# Cell 6: DCGAN Wrapper
# =====================================

import tensorflow as tf
from tensorflow.keras import optimizers

class DCGAN(tf.keras.Model):
    def __init__(self, generator, discriminator, config):
        super().__init__()
        self.G = generator
        self.D = discriminator
        self.latent_dim = config["latent_dim"]

        self.d_optimizer = optimizers.Adam(
            config["lr"], beta_1=config["beta_1"]
        )
        self.g_optimizer = optimizers.Adam(
            config["lr"], beta_1=config["beta_1"]
        )

        self.loss_fn = tf.keras.losses.BinaryCrossentropy()

    def compile(self):
        super().compile()

# =====================================
# Cell 7: Training Utilities
# =====================================

import numpy as np
import tensorflow as tf

def smooth_labels(y, smooth_factor=0.1):
    return y - smooth_factor + np.random.random(y.shape) * smooth_factor

def add_noise(images, std=0.05):
    return images + tf.random.normal(tf.shape(images), mean=0.0, stddev=std)

In [ ]:
# =====================================
# Cell 8: DCGAN Training Logic (FINAL VERSION)
# =====================================

import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf


# ------------------------------------------------
# Single training step
# ------------------------------------------------
def train_step(dcgan, real_images):
    batch_size = tf.shape(real_images)[0]

    # Sample latent noise
    noise = tf.random.normal((batch_size, dcgan.latent_dim))

    # =========================
    # 1️⃣ Train Discriminator
    # =========================
    with tf.GradientTape() as d_tape:
        fake_images = dcgan.G(noise, training=True)

        real_pred = dcgan.D(real_images, training=True)
        fake_pred = dcgan.D(fake_images, training=True)

        # Label smoothing
        real_labels = tf.ones((batch_size, 1)) * 0.9
        fake_labels = tf.zeros((batch_size, 1))

        d_loss_real = dcgan.loss_fn(real_labels, real_pred)
        d_loss_fake = dcgan.loss_fn(fake_labels, fake_pred)
        d_loss = d_loss_real + d_loss_fake

    d_grads = d_tape.gradient(d_loss, dcgan.D.trainable_variables)
    dcgan.d_optimizer.apply_gradients(
        zip(d_grads, dcgan.D.trainable_variables)
    )

    # =========================
    # 2️⃣ Train Generator
    # =========================
    with tf.GradientTape() as g_tape:
        fake_images = dcgan.G(noise, training=True)
        fake_pred = dcgan.D(fake_images, training=False)

        g_labels = tf.ones((batch_size, 1))
        g_loss = dcgan.loss_fn(g_labels, fake_pred)

    g_grads = g_tape.gradient(g_loss, dcgan.G.trainable_variables)
    dcgan.g_optimizer.apply_gradients(
        zip(g_grads, dcgan.G.trainable_variables)
    )

    return d_loss, g_loss


# ------------------------------------------------
# Full training loop
# ------------------------------------------------
def train_dcgan(dcgan, train_dataset, epochs, checkpoint_dir):
    os.makedirs(checkpoint_dir, exist_ok=True)

    history = []

    print("\n🚀 DCGAN TRAINING STARTED")
    print("=" * 75)

    for epoch in range(1, epochs + 1):
        start_time = time.time()

        d_losses = []
        g_losses = []

        for real_images in train_dataset:
            d_loss, g_loss = train_step(dcgan, real_images)
            d_losses.append(d_loss.numpy())
            g_losses.append(g_loss.numpy())

        mean_d = float(np.mean(d_losses))
        mean_g = float(np.mean(g_losses))
        elapsed = time.time() - start_time

        # -------- PRINT PER-EPOCH OUTPUT --------
        print(
            f"Epoch [{epoch:03d}/{epochs}] | "
            f"D Loss: {mean_d:.4f} | "
            f"G Loss: {mean_g:.4f} | "
            f"Time: {elapsed:.1f}s"
        )

        # -------- SAVE CHECKPOINTS --------
        if epoch % 10 == 0:
            dcgan.G.save(
                os.path.join(checkpoint_dir, f"G_epoch_{epoch:03d}.keras")
            )
            dcgan.D.save(
                os.path.join(checkpoint_dir, f"D_epoch_{epoch:03d}.keras")
            )

            # Latest models (easy reload)
            dcgan.G.save(os.path.join(checkpoint_dir, "G_latest.keras"))
            dcgan.D.save(os.path.join(checkpoint_dir, "D_latest.keras"))

            print(f"✅ Checkpoints saved at epoch {epoch}")

        history.append({
            "epoch": epoch,
            "d_loss": mean_d,
            "g_loss": mean_g,
            "time_sec": elapsed
        })

    print("=" * 75)
    print("✅ DCGAN TRAINING COMPLETED")

    history_df = pd.DataFrame(history)
    history_df.to_csv(
        os.path.join(checkpoint_dir, "training_log.csv"),
        index=False
    )

    return history_df
generator = build_generator(latent_dim=CONFIG["latent_dim"])
discriminator = build_discriminator(
    img_size=CONFIG["img_size"],
    channels=CONFIG["channels"]
)

dcgan = DCGAN(generator, discriminator, CONFIG)

train_dataset = load_plantvillage_dataset(
    img_size=CONFIG["img_size"],
    batch_size=CONFIG["batch_size"]
)

history_df = train_dcgan(
    dcgan=dcgan,
    train_dataset=train_dataset,
    epochs=CONFIG["epochs"],
    checkpoint_dir=CONFIG["checkpoint_dir"]
)

history_df.tail()


In [ ]:
# =====================================
# Cell 9: Inception Score (IS) – COMPLETE SINGLE CELL
# =====================================

import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input


# ------------------------------------------------
# 1⍡ Paths & parameters
# ------------------------------------------------
EVAL_DIR = r"/content/drive/MyDrive/DCGANProject/evaluation"
CHECKPOINT_DIR = r"/content/drive/MyDrive/DCGANProject/checkpoints"
DATASET_ROOT = r"/content/drive/MyDrive/DCGANProject/ExampleDataDir/PartialPlantExamples"
os.makedirs(EVAL_DIR, exist_ok=True)

N_EVAL = 256      # number of generated images
SPLITS = 10       # standard value for IS
INCEPTION_BATCH_SIZE = 16 # New parameter for batching InceptionV3 inference (Reduced further to avoid OOM)


# ------------------------------------------------
# 2⍡ Load trained Generator
# ------------------------------------------------
g_path = os.path.join(CHECKPOINT_DIR, "G_latest.keras")
if not os.path.exists(g_path):
    raise RuntimeError("❌ Generator not found. Train DCGAN first.")

generator = tf.keras.models.load_model(g_path, compile=False)
latent_dim = generator.input_shape[1]

print("✅ Generator loaded")
print("Latent dimension:", latent_dim)


# ------------------------------------------------
# 3⍡ Load InceptionV3 classifier (REQUIRED for IS)
# ------------------------------------------------
inception_model = InceptionV3(
    include_top=True,      #   must be True (softmax output)
    weights="imagenet"
)

print("✅ InceptionV3 classifier loaded")


# ------------------------------------------------
# 4⍡ Generate fake images
# ------------------------------------------------
def generate_fake_images(n):
    z = tf.random.normal((n, latent_dim))
    fake = generator(z, training=False)
    fake = (fake + 1.0) / 2.0   # [-1,1] → [0,1]
    return fake


# ------------------------------------------------
# 5⍡ Compute Inception Score (CORRECT)
# ------------------------------------------------
def compute_inception_score(images, splits=10, batch_size=32):
    """
    images: Tensor [N, H, W, 3] in range [0,1]
    returns: (IS_mean, IS_std)
    """

    # Predict class probabilities in batches to avoid OOM
    all_preds = []
    num_images = images.shape[0]
    for i in range(0, num_images, batch_size):
        batch_images = images[i:min(i + batch_size, num_images)]

        # Resize for InceptionV3
        batch_images = tf.image.resize(batch_images, (299, 299))

        # Preprocess for InceptionV3
        batch_images = preprocess_input(batch_images * 255.0)

        preds = inception_model(batch_images, training=False).numpy()
        all_preds.append(preds)

    preds = np.concatenate(all_preds, axis=0)

    # Safety normalization
    preds = preds / np.sum(preds, axis=1, keepdims=True)

    scores = []
    N = preds.shape[0]
    split_size = N // splits

    for i in range(splits):
        part = preds[i * split_size:(i + 1) * split_size]
        p_y = np.mean(part, axis=0, keepdims=True)

        kl = part * (np.log(part + 1e-10) - np.log(p_y + 1e-10))
        kl = np.sum(kl, axis=1)

        scores.append(np.exp(np.mean(kl)))

    return float(np.mean(scores)), float(np.std(scores))


# ------------------------------------------------
# 6⍡ RUN Inception Score
# ------------------------------------------------
fake_images = generate_fake_images(N_EVAL)

is_mean, is_std = compute_inception_score(
    fake_images,
    splits=SPLITS,
    batch_size=INCEPTION_BATCH_SIZE # Pass the batch size for inference
)

print(f"📊 Inception Score = {is_mean:.3f} ± {is_std:.3f}")


# ------------------------------------------------
# 7⍡ Save results
# ------------------------------------------------
df = pd.DataFrame([{
    "InceptionScore_mean": is_mean,
    "InceptionScore_std": is_std,
    "num_images": N_EVAL
}])

df.to_csv(os.path.join(EVAL_DIR, "inception_score.csv"), index=False)

print("✅ Inception Score saved to evaluation/inception_score.csv")

In [ ]:
# =====================================
# Cell 10: Load Trained Generator / Discriminator
# =====================================

import os
import tensorflow as tf

def load_trained_models(checkpoint_dir, epoch=None):
    """
    Loads trained Generator and Discriminator.
    If epoch is None, loads the latest available model.
    """
    if epoch is None:
        g_path = os.path.join(checkpoint_dir, "G_latest.keras")
        d_path = os.path.join(checkpoint_dir, "D_latest.keras")
    else:
        g_path = os.path.join(checkpoint_dir, f"G_epoch_{epoch:03d}.keras")
        d_path = os.path.join(checkpoint_dir, f"D_epoch_{epoch:03d}.keras")

    if not os.path.exists(g_path) or not os.path.exists(d_path):
        raise FileNotFoundError("Checkpoint files not found.")

    generator = tf.keras.models.load_model(g_path, compile=False)
    discriminator = tf.keras.models.load_model(d_path, compile=False)

    print("✅ Models loaded successfully")
    return generator, discriminator


In [ ]:
# =====================================
# Cell 11: Visualization (FIXED & SAFE)
# =====================================

import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd


FIG_DIR = r"C:\Users\devar\DCGANProject\figures"
CHECKPOINT_DIR =  r"C:\Users\devar\DCGANProject\checkpoints"
os.makedirs(FIG_DIR, exist_ok=True)


# ------------------------------------------------
# 1️⃣ Load history safely
# ------------------------------------------------
if "history_df" not in globals():
    log_path = os.path.join(CHECKPOINT_DIR, "training_log.csv")
    if os.path.exists(log_path):
        history_df = pd.read_csv(log_path)
        print("✅ Loaded training_log.csv")
    else:
        raise RuntimeError(
            "❌ history_df not found. Run Cell-8B (training) first."
        )


# ------------------------------------------------
# 2️⃣ Load generator safely
# ------------------------------------------------
if "generator" not in globals():
    g_path = os.path.join(CHECKPOINT_DIR, "G_latest.keras")
    if os.path.exists(g_path):
        generator = tf.keras.models.load_model(g_path, compile=False)
        print("✅ Loaded trained Generator")
    else:
        raise RuntimeError(
            "❌ Generator not found. Train model or load checkpoints."
        )


# ------------------------------------------------
# 3️⃣ Plot Training Loss Curves
# ------------------------------------------------
def plot_training_losses(history_df):
    plt.figure(figsize=(8, 5))
    plt.plot(history_df["epoch"], history_df["d_loss"], label="Discriminator Loss")
    plt.plot(history_df["epoch"], history_df["g_loss"], label="Generator Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("DCGAN Training Losses")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, "dcgan_loss_curve.png"))
    plt.show()


# ------------------------------------------------
# 4️⃣ Generate & Display Synthetic Images
# ------------------------------------------------
def generate_and_show_images(generator, latent_dim, n=16):
    noise = tf.random.normal((n, latent_dim))
    fake_images = generator(noise, training=False)

    # Rescale [-1,1] → [0,1]
    fake_images = (fake_images + 1.0) / 2.0

    grid_size = int(np.sqrt(n))
    plt.figure(figsize=(6, 6))

    for i in range(n):
        plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(fake_images[i])
        plt.axis("off")

    plt.suptitle("Generated Crop Leaf Images")
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, "generated_samples.png"))
    plt.show()


# ------------------------------------------------
# 5️⃣ Run Visualizations (FIXED)
# ------------------------------------------------

latent_dim = generator.input_shape[1]  # 🔑 infer automatically

plot_training_losses(history_df)

generate_and_show_images(
    generator=generator,
    latent_dim=latent_dim,
    n=16
)



In [ ]:
# =====================================
# Cell 12: Inference (Save ALL Images Individually)
# =====================================

import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------
# Paths
# ------------------------------------------------
# Use paths from the global CONFIG for consistency
CHECKPOINT_DIR = CONFIG["checkpoint_dir"]
OUTPUT_DIR = os.path.join(CONFIG["data_dir"].split("/")[0], CONFIG["data_dir"].split("/")[1], CONFIG["data_dir"].split("/")[2], CONFIG["data_dir"].split("/")[3], CONFIG["data_dir"].split("/")[4], "inference_outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ------------------------------------------------
# 1⍡ Load trained Generator
# ------------------------------------------------
g_path = os.path.join(CHECKPOINT_DIR, "G_latest.keras")

if not os.path.exists(g_path):
    raise RuntimeError(
        "❌ Trained Generator not found. Run Cell-8B (training) first."
    )

generator = tf.keras.models.load_model(g_path, compile=False)
print("✅ Generator loaded successfully")


# ------------------------------------------------
# 2⍡ Infer latent dimension automatically
# ------------------------------------------------
latent_dim = generator.input_shape[1]
print("Latent dimension:", latent_dim)


# ------------------------------------------------
# 3⍡ Generate synthetic images
# ------------------------------------------------
def generate_images(generator, n=64):
    print("Number of Images Generated =", n)

    noise = tf.random.normal((n, latent_dim))
    fake_images = generator(noise, training=False)

    # Rescale from [-1,1] → [0,1]
    fake_images = (fake_images + 1.0) / 2.0
    return fake_images


# ------------------------------------------------
# 4⍡ Save ALL images individually
# ------------------------------------------------
def save_all_images(images, output_dir, prefix="synthetic_leaf"):
    num_images = images.shape[0]

    for i in range(num_images):
        file_path = os.path.join(
            output_dir, f"{prefix}_{i+1:03d}.png"
        )
        plt.imsave(file_path, images[i])

    print(f"✅ Saved {num_images} images to '{output_dir}'")


# ------------------------------------------------
# 5⍡ RUN INFERENCE
# ------------------------------------------------
N_IMAGES = 64

synthetic_images = generate_images(
    generator=generator,
    n=N_IMAGES
)

save_all_images(
    images=synthetic_images,
    output_dir=OUTPUT_DIR,
    prefix="synthetic_leaf"
)

print("🎉 Inference complete")

In [ ]:
# =====================================
# Cell 13: Streamlit App – Leaf Disease Image Generator
# =====================================

!pip install streamlit
import streamlit as st
import tensorflow as tf
import numpy as np
import os
from PIL import Image

st.set_page_config(page_title="Leaf Disease GAN", layout="centered")
# ------------------------------------------------
# Configuration
# ------------------------------------------------
CHECKPOINT_DIR = r"/content/drive/MyDrive/DCGANProject/checkpoints"
OUTPUT_DIR = r"/content/drive/MyDrive/DCGANProject/streamlit_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ------------------------------------------------
# Load Generator (cached)
# ------------------------------------------------
@st.cache_resource
def load_generator():
    model_path = os.path.join(CHECKPOINT_DIR, "G_latest.keras")
    if not os.path.exists(model_path):
        st.error("❌ Generator model not found. Train DCGAN first.")
        st.stop()
    return tf.keras.models.load_model(model_path, compile=False)


generator = load_generator()
latent_dim = generator.input_shape[1]


# ------------------------------------------------
# Image generation function
# ------------------------------------------------
def generate_images(n):
    z = tf.random.normal((n, latent_dim))
    fake_images = generator(z, training=False)
    fake_images = (fake_images + 1.0) / 2.0  # [-1,1] → [0,1]
    return fake_images.numpy()


# ------------------------------------------------
# Streamlit UI
# ------------------------------------------------


st.title("🌱 Crop Leaf Disease Image Generator")
st.markdown(
    "Generate **synthetic crop leaf disease images** using a trained **DCGAN**."
)

num_images = st.slider(
    "Number of images to generate",
    min_value=1,
    max_value=64,
    value=16
)

generate_btn = st.button("Generate Images")


# ------------------------------------------------
# Generate & display images
# ------------------------------------------------
if generate_btn:
    st.info("Generating images...")
    images = generate_images(num_images)

    cols = st.columns(4)
    for i, img in enumerate(images):
        img_pil = Image.fromarray((img * 255).astype(np.uint8))
        save_path = os.path.join(OUTPUT_DIR, f"leaf_{i+1:03d}.png")
        img_pil.save(save_path)

        cols[i % 4].image(img_pil, caption=f"Image {i+1}", use_container_width=True)

    st.success(f"✅ {num_images} images generated and saved to `{OUTPUT_DIR}/`")

In [ ]:
# =====================================
# Cell 13: Streamlit App – Leaf Disease Image Generator
# =====================================

# NOTE: To run this Streamlit app, you need to save this code to a .py file
# and then execute it from the Colab terminal using `streamlit run <filename.py>`.
# A tunneling service like `localtunnel` or `ngrok` is also needed to expose
# the app to the internet from Colab.

# We will handle the installation and running in subsequent steps.

import streamlit as st
import tensorflow as tf
import numpy as np
import os
from PIL import Image

st.set_page_config(page_title="Leaf Disease GAN", layout="centered")
# ------------------------------------------------
# Configuration
# ------------------------------------------------
CHECKPOINT_DIR = r"/content/drive/MyDrive/DCGANProject/checkpoints"
OUTPUT_DIR = r"/content/drive/MyDrive/DCGANProject/streamlit_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ------------------------------------------------
# Load Generator (cached)
# ------------------------------------------------
@st.cache_resource
def load_generator():
    model_path = os.path.join(CHECKPOINT_DIR, "G_latest.keras")
    if not os.path.exists(model_path):
        st.error("❌ Generator model not found. Train DCGAN first.")
        st.stop()
    return tf.keras.models.load_model(model_path, compile=False)


generator = load_generator()
latent_dim = generator.input_shape[1]


# ------------------------------------------------
# Image generation function
# ------------------------------------------------
def generate_images(n):
    z = tf.random.normal((n, latent_dim))
    fake_images = generator(z, training=False)
    fake_images = (fake_images + 1.0) / 2.0  # [-1,1] → [0,1]
    return fake_images.numpy()


# ------------------------------------------------
# Streamlit UI
# ------------------------------------------------


st.title("🌱 Crop Leaf Disease Image Generator")
st.markdown(
    "Generate **synthetic crop leaf disease images** using a trained **DCGAN**."
)

num_images = st.slider(
    "Number of images to generate",
    min_value=1,
    max_value=64,
    value=16
)

generate_btn = st.button("Generate Images")


# ------------------------------------------------
# Generate & display images
# ------------------------------------------------
if generate_btn:
    st.info("Generating images...")
    images = generate_images(num_images)

    cols = st.columns(4)
    for i, img in enumerate(images):
        img_pil = Image.fromarray((img * 255).astype(np.uint8))
        save_path = os.path.join(OUTPUT_DIR, f"leaf_{i+1:03d}.png")
        img_pil.save(save_path)

        cols[i % 4].image(img_pil, caption=f"Image {i+1}", use_container_width=True)

    st.success(f"✅ {num_images} images generated and saved to `{OUTPUT_DIR}/`")

In [ ]:
!npm install -g localtunnel

In [ ]:
%%writefile app.py

import streamlit as st
import tensorflow as tf
import numpy as np
import os
from PIL import Image

st.set_page_config(page_title="Leaf Disease GAN", layout="centered")
# ------------------------------------------------
# Configuration
# ------------------------------------------------
CHECKPOINT_DIR = r"/content/drive/MyDrive/DCGANProject/checkpoints"
OUTPUT_DIR = r"/content/drive/MyDrive/DCGANProject/streamlit_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ------------------------------------------------
# Load Generator (cached)
# ------------------------------------------------
@st.cache_resource
def load_generator():
    model_path = os.path.join(CHECKPOINT_DIR, "G_latest.keras")
    if not os.path.exists(model_path):
        st.error("❌ Generator model not found. Train DCGAN first.")
        st.stop()
    return tf.keras.models.load_model(model_path, compile=False)


generator = load_generator()
latent_dim = generator.input_shape[1]


# ------------------------------------------------
# Image generation function
# ------------------------------------------------
def generate_images(n):
    z = tf.random.normal((n, latent_dim))
    fake_images = generator(z, training=False)
    fake_images = (fake_images + 1.0) / 2.0  # [-1,1] → [0,1]
    return fake_images.numpy()


# ------------------------------------------------
# Streamlit UI
# ------------------------------------------------


st.title("🌱 Crop Leaf Disease Image Generator")
st.markdown(
    "Generate **synthetic crop leaf disease images** using a trained **DCGAN**."
)

num_images = st.slider(
    "Number of images to generate",
    min_value=1,
    max_value=64,
    value=16
)

generate_btn = st.button("Generate Images")


# ------------------------------------------------
# Generate & display images
# ------------------------------------------------
if generate_btn:
    st.info("Generating images...")
    images = generate_images(num_images)

    cols = st.columns(4)
    for i, img in enumerate(images):
        img_pil = Image.fromarray((img * 255).astype(np.uint8))
        save_path = os.path.join(OUTPUT_DIR, f"leaf_{i+1:03d}.png")
        img_pil.save(save_path)

        cols[i % 4].image(img_pil, caption=f"Image {i+1}", use_container_width=True)

    st.success(f"✅ {num_images} images generated and saved to `{OUTPUT_DIR}/`")

In [ ]:
import subprocess
import time
import threading

def run_streamlit():
    # Run Streamlit in a separate process
    process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.enableCORS", "false", "--server.enableXsrfProtection", "false"])
    print("Streamlit app is running on port 8501...")
    process.wait() # Keep the process alive

def run_localtunnel():
    # Give Streamlit a moment to start
    time.sleep(10)
    print("Starting localtunnel...")
    # Expose the Streamlit port 8501 via localtunnel
    subprocess.run(["lt", "--port", "8501"])

# Start Streamlit in a separate thread
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.start()

# Start localtunnel in the main thread (or another thread if preferred)
run_localtunnel()

In [ ]:
# =====================================
# Cell 15: Monitoring & Versioning
# =====================================

import os
import yaml
import json
from datetime import datetime
import pandas as pd


# ------------------------------------------------
# Configuration
# ------------------------------------------------
REGISTRY_DIR = r"/content/drive/MyDrive/DCGANProject/model_registry"
LOG_DIR = r"/content/drive/MyDrive/DCGANProject/monitoring_logs"
EVAL_DIR = r"/content/drive/MyDrive/DCGANProject/evaluation"
CHECKPOINT_DIR = r"/content/drive/MyDrive/DCGANProject/checkpoints"

os.makedirs(REGISTRY_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

REGISTRY_FILE = os.path.join(REGISTRY_DIR, "model_registry.yaml")
USAGE_LOG_FILE = os.path.join(LOG_DIR, "usage_log.csv")


# ------------------------------------------------
# 1️⃣ Initialize Model Registry (if not exists)
# ------------------------------------------------
if not os.path.exists(REGISTRY_FILE):
    with open(REGISTRY_FILE, "w") as f:
        yaml.safe_dump({"models": []}, f)

print("✅ Model registry ready")


# ------------------------------------------------
# 2️⃣ Register a new trained model version
# ------------------------------------------------
def register_model(
    model_name,
    version,
    generator_path,
    training_config,
    metrics=None
):
    with open(REGISTRY_FILE, "r") as f:
        registry = yaml.safe_load(f)

    entry = {
        "model_name": model_name,
        "version": version,
        "generator_path": generator_path,
        "registered_at": datetime.now().isoformat(),
        "training_config": training_config,
        "metrics": metrics or {}
    }

    registry["models"].append(entry)

    with open(REGISTRY_FILE, "w") as f:
        yaml.safe_dump(registry, f)

    print(f"✅ Model {model_name} v{version} registered")


# ------------------------------------------------
# 3️⃣ Load latest evaluation metrics
# ------------------------------------------------
def load_latest_metrics():
    metrics = {}

    fid_file = os.path.join(EVAL_DIR, "gan_metrics.csv")
    is_file = os.path.join(EVAL_DIR, "inception_score.csv")

    if os.path.exists(fid_file):
        fid_df = pd.read_csv(fid_file)
        metrics["FID"] = float(fid_df.iloc[-1]["FID"])

    if os.path.exists(is_file):
        is_df = pd.read_csv(is_file)
        metrics["IS_mean"] = float(is_df.iloc[-1]["InceptionScore_mean"])
        metrics["IS_std"] = float(is_df.iloc[-1]["InceptionScore_std"])

    return metrics


# ------------------------------------------------
# 4️⃣ Register current model automatically
# ------------------------------------------------
MODEL_NAME = "DCGAN_Leaf_Disease"
MODEL_VERSION = "1.0"

training_config = {
    "latent_dim": 100,
    "image_size": 64,
    "optimizer": "Adam",
    "learning_rate": 0.0002,
    "beta_1": 0.5,
    "epochs": 100
}

metrics = load_latest_metrics()

register_model(
    model_name=MODEL_NAME,
    version=MODEL_VERSION,
    generator_path=os.path.join(CHECKPOINT_DIR, "G_latest.keras"),
    training_config=training_config,
    metrics=metrics
)


# ------------------------------------------------
# 5️⃣ Usage logging (Streamlit / API)
# ------------------------------------------------
def log_usage(
    source="streamlit",
    num_images=0,
    crop=None,
    disease=None
):
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "source": source,
        "num_images": num_images,
        "crop": crop,
        "disease": disease
    }

    if not os.path.exists(USAGE_LOG_FILE):
        df = pd.DataFrame([log_entry])
    else:
        df = pd.read_csv(USAGE_LOG_FILE)
        df = pd.concat([df, pd.DataFrame([log_entry])], ignore_index=True)

    df.to_csv(USAGE_LOG_FILE, index=False)


# ------------------------------------------------
# 6️⃣ Example usage log (demo)
# ------------------------------------------------
log_usage(
    source="streamlit",
    num_images=16,
    crop="tomato",
    disease="blight"
)

print("✅ Usage logged")


# ------------------------------------------------
# 7️⃣ Read monitoring summary
# ------------------------------------------------
def monitoring_summary():
    if not os.path.exists(USAGE_LOG_FILE):
        print("No usage logs yet.")
        return

    df = pd.read_csv(USAGE_LOG_FILE)
    print("\n📊 Monitoring Summary")
    print("---------------------")
    print("Total requests :", len(df))
    print("Total images generated :", df["num_images"].sum())
    print("Most requested crop :", df["crop"].mode().values[0])
    print("Most requested disease :", df["disease"].mode().values[0])


monitoring_summary()
